In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Dense, Input, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model

In [2]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'dataset\jpn.txt'

# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 72
Number of unique output tokens: 1435
Max sequence length for inputs: 18
Max sequence length for outputs: 28


In [3]:
print(input_texts)
print(target_texts)

['Go.', 'Go.', 'Hi.', 'Hi.', 'Hi.', 'Hi.', 'Run.', 'Run.', 'Who?', 'Wow!', 'Wow!', 'Wow!', 'Wow!', 'Duck!', 'Duck!', 'Fire!', 'Fire!', 'Fire!', 'Help!', 'Help!', 'Hide.', 'Jump!', 'Jump!', 'Jump!', 'Jump!', 'Jump!', 'Jump.', 'Jump.', 'Jump.', 'Jump.', 'Stop!', 'Stop!', 'Wait!', 'Wait!', 'Wait!', 'Wait.', 'Wait.', 'Go on.', 'Go on.', 'Go on.', 'Go on.', 'Hello!', 'Hello!', 'Hello!', 'Hello.', 'Hello.', 'Hello.', 'Hurry!', 'I bit.', 'I bit.', 'I bit.', 'I see.', 'I see.', 'I see.', 'I see.', 'I see.', 'I see.', 'I see.', 'I see.', 'I try.', 'I try.', 'I try.', 'I try.', 'I try.', 'I won!', 'I won!', 'I won!', 'I won!', 'I won!', 'Oh no!', 'Oh no!', 'Oh no!', 'Oh no!', 'Oh no!', 'Oh no!', 'Relax.', 'Relax.', 'Relax.', 'Relax.', 'Shoot!', 'Smile.', 'Smile.', 'Buy it.', 'Cheers!', 'Fix it.', 'Freeze!', 'Get up.', 'Get up.', 'Get up.', 'Go now.', 'Got it!', 'Got it!', 'He ran.', 'He ran.', 'Hop in.', 'Hop in.', 'Hug me.', 'Hug me.', 'I know.', 'I know.', 'I left.', 'I lost.', 'I paid.', 'I q

In [5]:
print(input_characters)
print(target_characters)

[' ', '!', '"', '$', '%', "'", ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
['\t', '\n', ' ', '%', '0', '1', '2', '3', '4', '5', '6', '8', '9', 'A', 'B', 'C', 'D', 'F', 'H', 'I', 'N', 'O', 'P', 'U', 'V', 'e', 'f', 'i', 'o', 'r', 'x', '—', '、', '。', '々', '「', '」', '〜', 'ぁ', 'あ', 'い', 'ぅ', 'う', 'ぇ', 'え', 'ぉ', 'お', 'か', 'が', 'き', 'ぎ', 'く', 'ぐ', 'け', 'げ', 'こ', 'ご', 'さ', 'ざ', 'し', 'じ', 'す', 'ず', 'せ', 'ぜ', 'そ', 'ぞ', 'た', 'だ', 'ち', 'っ', 'つ', 'づ', 'て', 'で', 'と', 'ど', 'な', 'に', 'ぬ', 'ね', 'の', 'は', 'ば', 'ぱ', 'ひ', 'び', 'ぴ', 'ふ', 'ぶ', 'ぷ', 'へ', 'べ', 'ぺ', 'ほ', 'ぼ', 'ぽ', 'ま', 'み', 'む', 'め', 'も', 'ゃ', 'や', 'ゅ', 'ゆ', 'ょ', 'よ', 'ら', 'り', 'る', 'れ', 'ろ', 'わ', 'を', 'ん', 'ァ', 'ア', 'ィ', 'イ', 'ゥ', 'ウ', 'ェ', 'エ', 'ォ', 'オ', 'カ', '

In [6]:

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype="float32",
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype="float32",
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype="float32",
)

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)
# Save model
model.save("s2s_model.keras")


Epoch 1/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 21s 145ms/step - accuracy: 0.6598 - loss: 2.0380 - val_accuracy: 0.6231 - val_loss: 2.0646
Epoch 2/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 16s 129ms/step - accuracy: 0.6770 - loss: 1.6337 - val_accuracy: 0.6370 - val_loss: 1.8324
Epoch 3/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 16s 127ms/step - accuracy: 0.7034 - loss: 1.5378 - val_accuracy: 0.6669 - val_loss: 1.7416
Epoch 4/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 16s 127ms/step - accuracy: 0.7326 - loss: 1.4201 - val_accuracy: 0.6924 - val_loss: 1.6697
Epoch 5/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 16s 131ms/step - accuracy: 0.7417 - loss: 1.3514 - val_accuracy: 0.6995 - val_loss: 1.5753
Epoch 6/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 16s 128ms/step - accuracy: 0.7515 - loss: 1.2938 - val_accuracy: 0.7174 - val_loss: 1.5085
Epoch 7/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 16s 129ms/step - accuracy: 0.7594 - loss: 1.2453 - val_accuracy: 0.7155 - val_loss: 1.4831
Epoch 8/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 16s 126ms/step - accuracy: 0.7647 -

In [8]:

# Define sampling models
# Restore the model and construct the encoder and decoder.
model = load_model("s2s_model.keras")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq, verbose=0)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value, verbose=0
        )

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence

for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)

-
Input sentence: Go.
Decoded sentence: 行け。

-
Input sentence: Go.
Decoded sentence: 行け。

-
Input sentence: Hi.
Decoded sentence: こんにちは！

-
Input sentence: Hi.
Decoded sentence: こんにちは！

-
Input sentence: Hi.
Decoded sentence: こんにちは！

-
Input sentence: Hi.
Decoded sentence: こんにちは！

-
Input sentence: Run.
Decoded sentence: 走れ。

-
Input sentence: Run.
Decoded sentence: 走れ。

-
Input sentence: Who?
Decoded sentence: 誰？

-
Input sentence: Wow!
Decoded sentence: おう！

-
Input sentence: Wow!
Decoded sentence: おう！

-
Input sentence: Wow!
Decoded sentence: おう！

-
Input sentence: Wow!
Decoded sentence: おう！

-
Input sentence: Duck!
Decoded sentence: 撃て！

-
Input sentence: Duck!
Decoded sentence: 撃て！

-
Input sentence: Fire!
Decoded sentence: 火事！

-
Input sentence: Fire!
Decoded sentence: 火事！

-
Input sentence: Fire!
Decoded sentence: 火事！

-
Input sentence: Help!
Decoded sentence: 助けてくれ！

-
Input sentence: Help!
Decoded sentence: 助けてくれ！

-
Input sentence: Hide.
Decoded sentence: 隠れろ。

-
Input senten